In [44]:
!pip install pm4py

In [45]:
# from google.colab import drive
# drive.mount("/content/drive")

In [46]:
import os
import pm4py
import graphviz
import pandas as pd
import datetime as dt
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as log_converter
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.visualization.petri_net import visualizer as pn_visualizer

In [47]:
sample = pd.read_csv("./sample1.csv")

In [48]:
sample.head()

,proc_ym,proc_ymd,userid,mcode,lecture_type,event,action,object_type,object_name,object_description,referrer_type,referrer_name,referrer_description,referrer_extensions_contentid,eventtime_kst,caliper_timestamp,data_source,yyyy,mm,dd
0,202108,20210809,6485fe71-8a0a-41d4-b76c-02a561f34339,NaN,NaN,SessionEvent,LoggedIn,SoftwareApplication,kr.hbstudy.apphbstudy,밀크T초등,NaN,NaN,NaN,NaN,2021-08-09 12:20:29.006,2021-08-09 12:20:29.006,e_session,2021,8,9
1,202108,20210809,6485fe71-8a0a-41d4-b76c-02a561f34339,NaN,NaN,SessionEvent,Paused,SoftwareApplication,kr.hbstudy.apphbstudy,밀크T초등,NaN,NaN,NaN,NaN,2021-08-09 12:26:05.250,2021-08-09 12:26:05.250,e_session,2021,8,9
2,202108,20210809,6485fe71-8a0a-41d4-b76c-02a561f34339,NaN,NaN,SessionEvent,Resumed,SoftwareApplication,kr.hbstudy.apphbstudy,밀크T초등,NaN,NaN,NaN,NaN,2021-08-09 12:51:45.020,2021-08-09 12:51:45.020,e_session,2021,8,9
3,202108,20210809,6485fe71-8a0a-41d4-b76c-02a561f34339,T0ME31U42019,T_TERMREVIEW,NavigationEvent,NavigatedTo,DigitalResource,준비하기,메뉴 인덱스 : 1,DigitalResource,인트로,메뉴 인덱스 : 0,T0ME31U42019,2021-08-09 12:53:45.834,2021-08-09 12:53:45.834,e_navigation,2021,8,9
4,202108,20210809,6485fe71-8a0a-41d4-b76c-02a561f34339,T0ME31U42019,T_TERMREVIEW,NavigationEvent,NavigatedTo,DigitalResource,개념 맛보기,메뉴 인덱스 : 2,DigitalResource,준비하기,메뉴 인덱스 : 1,T0ME31U42019,2021-08-09 12:54:00.839,2021-08-09 12:54:00.839,e_navigation,2021,8,9


In [49]:
case_list = []
case_number = 1

for ymd in sample["proc_ymd"].unique():
    target = sample[sample["proc_ymd"] == ymd]
    for case_add in range(len(target)):
        case_list.append(case_number)
    case_number += 1

sample["case_id"] = case_list

target = sample[["case_id", "eventtime_kst", "object_description"]].dropna()
eventlog = target.rename(
    columns={
        "case_id": "case:concept:name",
        "eventtime_kst": "time:timestamp",
        "object_description": "concept:name",
        "userid": "org:resource",
    }
)

In [50]:
target.head()

,case_id,eventtime_kst,object_description
0,1,2021-08-09 12:20:29.006,밀크T초등
1,1,2021-08-09 12:26:05.250,밀크T초등
2,1,2021-08-09 12:51:45.020,밀크T초등
3,1,2021-08-09 12:53:45.834,메뉴 인덱스 : 1
4,1,2021-08-09 12:54:00.839,메뉴 인덱스 : 2


In [53]:
# Convert the log to a suitable format for alpha miner
log = log_converter.apply(eventlog)

# Discover the process model using alpha miner
net, initial_marking, final_marking = alpha_miner.apply(log)

# Apply the Petri net visualizer
gviz = pn_visualizer.apply(net, initial_marking, final_marking)

# Convert the DOT source to a string
dot_string = gviz.source

# Render the modified DOT file using Graphviz
graph = graphviz.Source(dot_string)
graph

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [52]:
eventlog["time:timestamp"] = eventlog["time:timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
log = log_converter.apply(eventlog)

dfg = dfg_discovery.apply(log)
gviz = dfg_visualization.apply(
    dfg, log=log, variant=dfg_visualization.Variants.PERFORMANCE
)

dot_string = gviz.source

# Render the modified Dot file using graphviz
graph = graphviz.Source(dot_string)
graph

NameError: name 'dfg_visualization' is not defined

In [ ]:
eventlog["time:timestamp"] = eventlog["time:timestamp"].apply(
    lambda x: pd.to_datetime(x)
)
filtered = pm4py.filter_time_range(
    eventlog, dt.datetime(2010, 12, 30), dt.datetime(2010, 12, 31), mode="events"
)


log = log_converter.apply(eventlog)

dfg = dfg_discovery.apply(log)
gviz = dfg_visualization.apply(
    dfg, log=log, variant=dfg_visualization.Variants.FREQUENCY
)

dot_string = gviz.source

# Render the modified DOT file using Graphviz
graph = graphviz.Source(dot_string)
graph

In [ ]:
log_1 = log_converter.apply(log)
filtered = pm4py.get_event_attribute_values(log_1, "concept:name", {"밀크T초등"})

In [ ]:
eventlog["time:timestamp"] = eventlog["time:timestamp"].apply(
    lambda x: pd.to_datetime(x)
)

log = log_converter.apply(eventlog)

dfg = dfg_discovery.apply(log)
gviz = dfg_visualization.apply(
    dfg, log=log, variant=dfg_visualization.Variants.FREQUENCY
)

dot_string = gviz.source

# Render the modified DOT file using Graphviz
graph = graphviz.Source(dot_string)
graph